# A quck 10 Minutes introduction to Pandas

Credit: this is a notebook cribbed from [the official "10 Minutes to Pandas"](http://pandas.pydata.org/pandas-docs/stable/10min.html).

## Pandas: tools for data analysis in Python

* emphasis on tabular data (csv and the like)
* database/spreadsheet-like functionality
* rich support for mixed data (numpy is for homogeneous arrays)
* integrates cleanly with numpy and matplotlib

<center>
<a href="http://pandas.pydata.org" target="_blank"> 
<img src="files/pandas-book.jpg"></a>
</center>

There is also a wonderful [set of free tutorials](https://bitbucket.org/hrojas/learn-pandas) on Pandas.

In [ ]:
import pandas as pd

# Let's also load the plotting/numerical tools
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

# make figures a bit larger by default
plt.rcParams['figure.figsize'] = (10,6)

## A quick taste of Pandas

Consider a standard stocks file you downloaded from [Yahoo Finance](http://finance.yahoo.com/q/hp?s=AAPL+Historical+Prices) (note that Pandas actually [ships with a utility to get that data for you](http://pandas.pydata.org/pandas-docs/stable/remote_data.html#remote-data-yahoo)):

In [ ]:
!head AAPL.csv

In [ ]:
stock = pd.read_csv('AAPL.csv')
stock.head()

In [ ]:
stock.index = stock.pop('Date')
stock.head()

In [ ]:
stock.sort(inplace=True)
stock.head()

In [ ]:
stock['Adj Close'].plot();

## Object creation

See the [Data Structure Intro](http://pandas.pydata.org/pandas-docs/stable/dsintro.html#dsintro) section for more details.

Creating a Series by passing a list of values, letting pandas create a default integer index

In [ ]:
s = pd.Series([1, 3, 5, np.nan, 6, 8])
s

Creating a DataFrame by passing a numpy array, with a datetime index and labeled columns.

In [ ]:
dates = pd.date_range('20130101', periods=6)
dates

In [ ]:
rng = np.random.RandomState()

df = pd.DataFrame(rng.normal(size=(6, 4)), index=dates, columns=('rainfall', 'temp', 'humidity', 'windchill'))
df

## Viewing Data

In [ ]:
df.head()

In [ ]:
df.tail(3)

Display the index,columns, and the underlying numpy data

In [ ]:
df.index

In [ ]:
df.columns

In [ ]:
df.values

Describe shows a quick statistic summary of your data

In [ ]:
df.describe()

Transposing your data

In [ ]:
df.T

Sorting by an axis

In [ ]:
df.sort_index(ascending=False)

Sorting by values

In [ ]:
df.sort(columns='temp')

## Selection

**Note** While many of the NumPy access methods work on DataFrames, use the pandas-specific data access methods, `.at`, `.iat`, `.loc`, `.iloc` and `.ix`.

See the [Indexing section](http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing) and below.

### Getting

Selecting a single column, which yields a `Series`, equivalent to `df.A`

In [ ]:
df['rainfall']

Selecting via `[]`, which slices the rows.

In [ ]:
df[:3]

### Selection by Label

In [ ]:
df.loc['20130101':'20130103']

See more in [Selection by Label](http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-label).

For getting a cross section using a label

In [ ]:
df.loc[dates[0]]

Selecting on a multi-axis by label

In [ ]:
df.loc[:, ['rainfall', 'temp']]

### Selection by Position

Select via the position of the passed integers—esentially equivalent to NumPy indexing.

In [ ]:
df.iloc[3]

By integer slices, acting similar to numpy/python

In [ ]:
df.iloc[3:5, 0:2]

By lists of integer position locations, similar to the numpy/python style

In [ ]:
df.iloc[[1,2,4], [0,2]]

For slicing rows or columns explicitly

In [ ]:
df.iloc[1:3, :]

### Boolean Indexing

In [ ]:
df

In [ ]:
df[df.rainfall > 0]

What happens if I select on the whole table?

In [ ]:
df[df > 0]

## Basic operations

Simple statistics

In [ ]:
df.mean()

And the same along a different axis:

In [ ]:
df.mean(axis=1)

Applying a function to the data

In [ ]:
print(df)
df.apply(np.cumsum)

In [ ]:
df.apply(lambda column: column.max() - column.min())

SQL-style operations like merging data

In [ ]:
left = pd.DataFrame({'subject': ['history', 'literature'], 'papers': [10, 20]})
right = pd.DataFrame({'subject': ['history', 'literature', 'science'], 'books': [4, 5, 9]})

In [ ]:
left

In [ ]:
right

In [ ]:
pd.merge(left, right, on='subject', how='outer')

Above, `how` can be:

  * left: use only keys from left frame (SQL: left outer join)
  * right: use only keys from right frame (SQL: right outer join)
  * outer: use union of keys from both frames (SQL: full outer join)
  * inner: use intersection of keys from both frames (SQL: inner join)

Append rows to a dataframe:

In [ ]:
#df = pd.DataFrame(np.arange(4 * 3).reshape((4, 3)), columns=['A','B','C'])
df

In [ ]:
#s = df.iloc[2]
#s

In [ ]:
df.append(df.iloc[0], ignore_index=True)

In [ ]:
row = pd.DataFrame([[0, 1, 2, 3]], columns=df.columns, index=['2015-08-19'])
df.append(row)

## Grouping

By “group by” we are referring to a process involving one or more of the following steps

- **Splitting** the data into groups based on some criteria
- **Applying** a function to each group independently
- **Combining** the results into a data structure

See the [Grouping docs](http://pandas.pydata.org/pandas-docs/stable/groupby.html#groupby) for more.

In [ ]:
df = pd.DataFrame({'A' : ['foo', 'bar', 'foo', 'bar', 'foo', 'bar', 'foo', 'foo'],
                   'B' : ['one', 'one', 'two', 'three', 'two', 'two', 'one', 'three'],
                   'C' : range(8),
                   'D' : range(10, 18)})
df

Grouping and then applying a function sum to the resulting groups.

In [ ]:
df.groupby('A').sum()

Grouping by multiple columns forms a hierarchical index, which we then apply the function.

In [ ]:
df.groupby(['A', 'B']).sum()

In [ ]:
g = df.groupby('A')

In [ ]:
g.*?

```
g.A            g.B            g.C            g.D            g.agg          g.aggregate    
g.all          g.any          g.apply        g.bfill        g.boxplot      g.corr         
g.corrwith     g.count        g.cov          g.cumcount     g.cummax       g.cummin       
g.cumprod      g.cumsum       g.describe     g.diff         g.dtypes       g.ffill        
g.fillna       g.filter       g.first        g.get_group    g.groups       g.head         
g.hist         g.idxmax       g.idxmin       g.indices      g.irow         g.last         
g.mad          g.max          g.mean         g.median       g.min          g.name         
g.ngroups      g.nth          g.ohlc         g.pct_change   g.plot         g.prod         
g.quantile     g.rank         g.resample     g.sem          g.shift        g.size         
g.skew         g.std          g.sum          g.tail         g.take         g.transform    
g.tshift       g.var          
```